In [1]:
import apogee.tools.read as apread
from apogee.tools import bitmask
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import window as wn
import os
import sys
import access_spectrum as acs
import reduce_dataset as rd
import polyfit as pf
import matplotlib
import apogee.spec.plot as splot
# Show plots in notebook
%pylab inline
# Read in red clump sample
data = apread.rcsample()
# Set random seed value
seedval = 4
# Start random seed
np.random.seed(seedval) #original was 17
# Set global plot font preferences
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

Populating the interactive namespace from numpy and matplotlib


/Users/Natalie/Code/apogee/apogee/spec/window.py:88: RuntimeWarning: invalid value encountered in greater
  mask= ((win > 0.)*(True-numpy.isnan(win))).astype('int')


In [2]:
# Choose output directories
direc = './red_clump/pickles/'
pdirec = './red_clump/fitplots/'
rdirec = './red_clump/residual_plots/' 

In [3]:
ngen = False
# Load in spectra
# Choose parameter space to slice in, with upper and lower limit
label = 'FE_H'
low = -0.5
up = -0.4
# Choose data file names
specsname = 'specs_{0}_u{1}_d{2}.pkl'.format(label,low,up)
errsname = 'errs_{0}_u{1}_d{2}.pkl'.format(label,low,up)
maskname = 'mask_{0}_u{1}_d{2}.pkl'.format(label,low,up)

sliceinfo = [label,low,up]
# Find data indices corresponding to out slice
sindx = rd.slice_data(data, sliceinfo)
sdata = data[sindx]

if os.path.isfile(direc+specsname) and os.path.isfile(direc+errsname) and os.path.isfile(direc+maskname) and ngen:
    specs = acs.pklread(direc+specsname)
    errs = acs.pklread(direc+errsname)
    maskdata = acs.pklread(direc+specsname)

elif not os.path.isfile(direc+specsname) or not os.path.isfile(direc+errsname) or not os.path.isfile(direc+maskname) or not ngen:
    # Get set of bitmasks
    maskdata = acs.get_spectra_ap(sdata,ext = 3, header = False,indx = 1)
    ints = [i for i in range(len(maskdata)) if not isinstance(maskdata[i],np.int16)]
    maskdata = maskdata[ints]
    maskdata = np.array(list(maskdata),dtype = np.int64)
    # Get spectra and spectra uncertainty after ASPCAP pipeline
    specs = acs.get_spectra_asp(sdata,header = False)
    specs = specs[ints]
    errs = acs.get_spectra_asp(sdata,ext = 2, header = False)
    errs = errs[ints]
    acs.pklwrite(direc+maskname,maskdata)
    acs.pklwrite(direc+errsname,errs)
    acs.pklwrite(direc+specsname,specs)

In [8]:
elemwindows

{'Al': (array([6275, 6276, 6278, 6279, 6280, 6281, 6282, 6283, 6284, 6285, 6286,
         6287, 6288, 6289, 6290, 6291, 6292, 6293, 6294, 6295, 6296, 6297,
         6298, 6299, 6300, 6301, 6302, 6303, 6304, 6305, 6306, 6307, 6308,
         6309, 6310, 6314, 6315, 6316, 6317, 6318, 6479, 6481, 6482, 6483,
         6484, 6485, 6486, 6487, 6488, 6489, 6490, 6491, 6492, 6493, 6494,
         6495, 6496, 6497]),),
 'C': (array([ 606,  608,  609,  612,  613,  614,  615,  616,  617,  618,  619,
          623,  624,  625,  628,  629,  630,  631,  632,  633,  634,  635,
          636,  638,  639,  640,  722,  725,  727,  728,  729,  730,  731,
          732,  733,  734,  735,  736,  737,  739,  740,  742,  744,  745,
          746,  747,  748,  749,  750,  752,  753,  755,  756,  757,  758,
          759,  760,  762,  765,  766,  770,  773,  774,  776,  777,  778,
          779,  780,  781,  782,  783,  784,  785,  788,  791,  943,  944,
          945,  946,  947,  948,  949,  950,  951,  952,  

In [7]:
# Find all pixels corresponding to any of the element windows so we can choose to plot those pixel fits
elems = ['Al','Ca','C','Fe','K','Mg','Mn','Na','Ni','N','O','Si','S','Ti','V']
totalw = np.zeros(len(specs[0]))
elemwindows = {}
for elem in elems:
    w = wn.read(elem,dr = 12,apStarWavegrid=False)
    elemwindows[elem] = np.where(w != 0)
    totalw += w

In [5]:
# Find fit residuals
# Set polynomial order
order = 2
# Choose bad pixel mask value
badcombpixmask= bitmask.badpixmask()+2**bitmask.apogee_pixmask_int("SIG_SKYLINE")
# Choose names for saving data files
ressname = 'rcresiduals_order{0}_{1}_u{2}_d{3}.pkl'.format(order,label,low,up)
gname = 'rcgerr_order{0}_{1}_u{2}_d{3}_seed{4}.pkl'.format(order,label,low,up,seedval)
paramname = 'fitparam_{0}_{1}_u{2}_d{3}.pkl'.format(order,label,low,up)
disname = 'discard_rcresiduals_order{0}_{1}_u{2}_d{3}.pkl'.format(order,label,low,up)
failname = 'fails_rcresiduals_order{0}_{1}_u{2}_d{3}.pkl'.format(order,label,low,up)
def rcfitplotname(pix):
    return 'pix{0}fit_order{1}_{2}_u{3}_d{4}.png'.format(pix,order,label,low,up)

# Choose whether to force regeneration of fit data
ngen = False
# Choose whether to generate some fit plots
doplot = False

# If regeneration unecessary and files exist, load them in
if os.path.isfile(direc+ressname) and os.path.isfile(direc+gname) and os.path.isfile(direc+paramname) and os.path.isfile(direc+disname) and ngen:
    ress = acs.pklread(direc+ressname)
    gerr = acs.pklread(direc+gname)

# If regeneration necessary or files don't exist, create them
elif not os.path.isfile(direc+ressname) or not os.path.isfile(direc+gname) or not os.path.isfile(direc+paramname) or not os.path.isfile(direc+disname) or not ngen:
    ps = []
    ress = []
    gerr = []
    discard = []
    fails = []
    for pix in range(len(specs[0])):
        # Find bad pixels
        mask,badspec = rd.pixmask_find(maskdata,pix,bitval = badcombpixmask)
        
        # Apply mask
        mdata = sdata[mask]
        mspecs = specs[mask]
        merrs = errs[mask]
        
        SNR = mspecs[:,pix]/merrs[:,pix]
        toogood = np.where(SNR > 200)
        merrs[:,pix][toogood] = mspecs[:,pix][toogood]/200.
        
        # Note how many spectra were discarded in the masking process
        discard.append('{0} of {1}'.format(len(sdata)-len(mask[0]),len(sdata)))
        
        # Create arrays in the independent variables
        Teff = np.arange(min(mdata['TEFF']),max(mdata['TEFF']))
        logg = np.arange(min(mdata['LOGG']),max(mdata['LOGG']),((max(mdata['LOGG']-min(mdata['LOGG'])))/len(Teff)))
        metals = np.arange(min(mdata['METALS']),max(mdata['METALS']),((max(mdata['METALS']-min(mdata['METALS'])))/len(Teff)))
        
        # Set up zero arrays to hold residuals so missing stars will vanish after weighting
        res = np.zeros(len(specs))
        erarr = np.zeros(len(specs))
        
        try: 
            # Find the fit parameters
            p = pf.regfit((mdata['TEFF'],mdata['LOGG'],mdata['FE_H']),mspecs[:,pix],err = merrs[:,pix],order = order)
            ps.append(p)
            
            # Calculate residuals from the fit
            res[mask] = mspecs[:,pix] - pf.poly(p,(mdata['TEFF'],mdata['LOGG'],mdata['METALS']),order = order)
            ress.append(res)
            
            # Find measurement noise
            for mval in mask[0]:
                erarr[mask[0][mval]] = np.random.normal(0,errs[mval][pix])
            gerr.append(erarr)
            
            if totalw[pix] != 0 and doplot:
                # Find points with/out high uncertainty that are not zero
                # This is to avoid large errorbars and reduce the plotting to a manageable scale
                rpl = np.where((merrs[:,pix] > 0.1) & (mspecs[:,pix] != 0))
                pl = np.where((merrs[:,pix] < 0.1) & (mspecs[:,pix] != 0))
                
                # Start figure
                plt.figure(figsize = (12,10))
                
                # First column - Temperature and flux uncertainty
                # Fit vs Temperature
                plt.subplot2grid((3,3),(0,0))
                plt.errorbar(mdata['TEFF'][rpl],mspecs[:,pix][rpl],yerr = merrs[:,pix][rpl],fmt='.')
                plt.plot(mdata['TEFF'][pl],mspecs[:,pix][pl],'.',alpha=0.2)
                plt.plot(Teff,pf.poly(p,(Teff,logg,metals),order = order),linewidth = 3)
                plt.xlabel('T_EFF')
                plt.ylabel('Pixel {0}'.format(pix))
                # Residuals vs Temperature
                plt.subplot2grid((3,3),(1,0))
                plt.plot(mdata['TEFF'],res,'.',alpha = 0.2)
                plt.xlabel('T_EFF')
                plt.ylabel('Fit residuals')
                # Residuals vs flux uncertainty
                plt.subplot2grid((3,3),(2,0))
                plt.semilogx(errs[:,pix],res,'.',alpha = 0.2)
                plt.xlabel('Uncertainty in Pixel {0}'.format(pix))
                plt.ylabel('Fit residuals')
                
                # Second column - surface gravity
                # Fit vs surface gravity
                plt.subplot2grid((3,3),(0,1))
                plt.errorbar(mdata['LOGG'][rpl],mspecs[:,pix][rpl],yerr = merrs[:,pix][rpl],fmt='.')
                plt.plot(mdata['LOGG'][pl],mspecs[:,pix][pl],'.',alpha = 0.2)
                plt.plot(logg,pf.poly(p,(Teff,logg,metals),order = order),linewidth = 3)
                plt.xlabel('LOG(G)')
                plt.ylabel('Pixel {0}'.format(pix))
                # Residuals vs surface gravity
                plt.subplot2grid((3,3),(1,1))
                plt.plot(mdata['LOGG'],res,'.',alpha = 0.2)
                plt.xlabel('LOG(G)')
                plt.ylabel('Fit residuals')
                
                # Third column - metals
                # Fit vs metals
                plt.subplot2grid((3,3),(0,2))
                plt.errorbar(mdata['METALS'][rpl],mspecs[:,pix][rpl],yerr = merrs[:,pix][rpl],fmt='.')
                plt.plot(mdata['METALS'][pl],mspecs[:,pix][pl],'.',alpha = 0.2)
                plt.plot(metals,pf.poly(p,(Teff,logg,metals),order = order),linewidth = 3)
                plt.xlabel('Metallicity')
                plt.ylabel('Pixel {0}'.format(pix))
                # Residuals vs metals
                plt.subplot2grid((3,3),(1,2))
                plt.plot(mdata['METALS'],res,'.',alpha = 0.2)
                plt.xlabel('Metallicity')
                plt.ylabel('Fit residuals')
                plt.tight_layout()
                
                # Save and close figure
                plt.savefig(pdirec+rcfitplotname(pix))
                plt.close()

        except LinAlgError:
            fails.append('Failure at pixel {0}'.format(pix))
            continue
    # Save to files
    ress = np.array(ress)
    gerr = np.array(gerr)
    acs.pklwrite(direc+failname,fails)
    acs.pklwrite(direc+disname,discard)
    acs.pklwrite(direc+paramname,ps)
    acs.pklwrite(direc+ressname,ress)
    acs.pklwrite(direc+gname,gerr)

In [ ]:
doplot = True
# Calculate weighted residual sum for each element
elems = ['Al','Ca','C','Fe','K','Mg','Mn','Na','Ni','N','O','Si','S','Ti','V']
ngen = False

# Record pixels corresponding to the different elements in a dictionary
elemwindow = {}
elemname = 'pixelwindows_order{0}_{1}_u{2}_d{3}.pkl'.format(order,label,low,up)

def rcresid_elemname(elem):
    return '{0}_residuals_order{1}_{2}_u{3}_d{4}.pkl'.format(elem,order,label,low,up)

def rcgerr_elemname(elem):
    return '{0}_rcgerr_order{1}_{2}_u{3}_d{4}_seed{5}.pkl'.format(elem,order,label,low,up,seedval)

# Cycle through all elements, checking for weighted residual files, creating them if necessary
for elem in elems:
    if os.path.isfile(direc+rcresid_elemname(elem)) and os.path.isfile(direc+rcgerr_elemname(elem)) and ngen:
        continue
    elif not os.path.isfile(direc+rcresid_elemname(elem)) or not os.path.isfile(direc+rcgerr_elemname(elem)) or not ngen:
        # Read in window associated with the element
        w = wn.read(elem,dr = 12,apStarWavegrid=False)
        th = wn.tophat(elem,dr=12,apStarWavegrid=False)
        # Record which pixels are relevant for this element
        elemwindow[elem] = np.where(th==True)[0]
        # Normalized weights
        nw = pf.normweights(w)[th]
        # Prepare lists to hold the weighted residual for each star for this element
        resids = []
        gresid = []
        for star in range(len(ress[0])):
            # Create weighted residual sum for each star
            resids.append(pf.genresidual(nw,ress[:,star][th]))
            gresid.append(pf.genresidual(nw,gerr[:,star][th]))
        resids = np.array(resids)
        gresid = np.array(gresid)
        # Save weighted residuals for all stars for this element
        acs.pklwrite(direc+rcresid_elemname(elem),resids)
        acs.pklwrite(direc+rcgerr_elemname(elem),gresid)

# Save pixel-element information if necessary
if not os.path.isfile(direc+elemname) or not ngen:
    acs.pklwrite(direc+elemname,elemwindow)

In [6]:
# Plot residuals for each element
doplot = True

# Name the plot for each element
def rcoutplotname(elem):
    return '{0}_residuals_order{1}_{2}_u{3}_d{4}_seed{5}.png'.format(elem,order,label,low,up,seedval)

# Make residual plots
if doplot:
    for elem in elems:
        resids = acs.pklread(direc+rcresid_elemname(elem))
        gresid = acs.pklread(direc+rcgerr_elemname(elem))
        plt.figure(figsize = (12,10))
        allow = np.where(np.isnan(gresid) == False)
        resids = resids[allow]
        gresid = gresid[allow]
        Rhist,Rbins = np.histogram(resids,bins = 50,range = (-0.1,0.1))
        Ghist,Gbins = np.histogram(gresid,bins = 50,range = (-0.1,0.1))
        # Plot residual histogram
        plt.bar(Rbins[:-1],Rhist/float(max(Rhist)),width = (Rbins[1]-Rbins[0]))
        # Plot flux uncertainty historgram
        plt.bar(Gbins[:-1],Ghist/float(max(Ghist)),width = (Gbins[1]-Gbins[0]),color = 'g',alpha = 0.7)
        plt.xlim(-0.1,0.1)
        plt.xlabel('Fit residual')
        plt.ylabel('Star Count Normalized to Maximum')
        plt.title('Uncertainty from $T_{eff}$, log($g$), $Z$ fit for '+elem)
        
        
        #plt.subplot(122)
        #plt.hist(gresid[allow],bins = 40,range = (-0.1,0.1),stacked=True)
        #plt.xlabel('Flux uncertainty'.format(elem))
        #plt.ylabel('Star Count')
        #plt.xlim(-0.1,0.1)
        #plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(rdirec+rcoutplotname(elem))

NameError: name 'rcresid_elemname' is not defined